In [1]:
import os, sys, gc, argparse
import numpy as np
from sklearn.model_selection import KFold

In [2]:

dataDir = '/tigress/ruairidh/superman/server/nature/data/simulations'
loadDir = '{0}/distributions'.format(dataDir)
saveDir = '{0}/model_results'.format(dataDir)

# In[3]:


parser = argparse.ArgumentParser()
parser.add_argument("-nc", type=int)
parser.add_argument("-nd", type=int)
parser.add_argument("-it", type=int)
args = parser.parse_args()
print(args)
nc = args.nc
nd = args.nd
it = args.it

beta = True

numDistributions = 2
fullCalc = True

In [3]:
def print_shape_fn(term, value, activated = False):
    if activated:
        print('{0} has shape: {1}'.format(term, value.shape))
        
def sample_from_data(train, trainLabels, numSamples):
    """Assuming 2 categories"""
    normalIdx = np.arange(trainLabels.shape[0])
    cat1Idx = normalIdx[np.where(trainLabels == 0)[0]]
    cat2Idx = normalIdx[np.where(trainLabels == 1)[0]]
    cat1Samples = np.random.permutation(cat1Idx)
    cat2Samples = np.random.permutation(cat2Idx)
    trainRed = np.vstack((train[cat1Samples[:numSamples]], train[cat2Samples[:numSamples]]))
    trainLabelRed = np.vstack((trainLabels[cat1Samples[:numSamples]], trainLabels[cat2Samples[:numSamples]]))
    print_shape_fn('trainRed', trainRed)
    print_shape_fn('trainLabelRed', trainLabelRed)
    return trainRed, trainLabelRed
     

In [4]:
class classification_model:
    def __init__(self, train, trainLabels, test, testLabels):
        self.train = train
        self.numTrainingSamples = self.train.shape[0]
        self.numDimensions = self.train.shape[1]
        self.trainLabels = trainLabels
        self.categories = np.unique(trainLabels)
        self.numCats = np.shape(self.categories)[0]
                                                                       
        self.test = test
        self.numTestingSamples = self.test.shape[0]
        self.testLabels = testLabels

    def find_test_distances(self, trained, test):
        """prototypes and tests should be samples by dimensions"""
        dists = (-2 * np.dot(test, trained.T) + np.sum(trained**2, axis=1) + np.sum(test**2, axis=1)[:, np.newaxis]).T
        #print('distances shape: {0}. Should be num categories by num test samples: {1}, {2}'.format(dists.shape, self.numCats, self.numTestingSamples))
        return dists
    
    def get_accuracy(self, predictions, testLabels):
        testLabels = np.squeeze(testLabels)
        difference = np.absolute(predictions - testLabels)
        accuracy = (self.numTestingSamples - np.sum(difference)) / self.numTestingSamples
        return accuracy
    
    
    
class prototype_model(classification_model):
    def __init__(self, train, trainLabels, test, testLabels):
        classification_model.__init__(self, train, trainLabels, test, testLabels)
        self.trained = self.form_prototype_matrix()
        self.testDistances = self.find_test_distances(self.trained, self.test)
        self.exponentMatrix = self.create_exponent_matrix()
        #print('exponentMatrix: {0}'.format(self.exponentMatrix))
        self.LL = self.score_model_LogSumExp()
        self.predictions = self.generate_predictions(self.test)
        
        self.accuracy = self.get_accuracy(self.predictions, self.testLabels)
        
    def form_prototype_matrix(self):
        means = []
        for cat in self.categories:
            subsample = self.train[np.where(self.trainLabels == cat)[0]]
            #print('for category {0}, sample matrix has this shape: {1}'.format(cat, subsample.shape))
            means.append(np.mean(subsample, axis = 0))
        meanArray = np.vstack(means)
        #print('meanArray: {0}'.format(meanArray))
        #print('meanArray has this shape: {0}'.format(meanArray.shape))
        return meanArray

    def create_exponent_matrix(self):
        """returns negated matrix"""
        candidateIdx = np.squeeze(self.testLabels), np.arange(len(self.testLabels))
        #print(candidateIdx)
        candidates = self.testDistances[candidateIdx]
        exponentMatrix = self.testDistances - candidates
        #print('exponent matrix has shape: {0}'.format(exponentMatrix.shape))
        return (- exponentMatrix)
    
    
    def score_model_LogSumExp(self):
        # should be cats by samples
        maxTerm = np.max(self.exponentMatrix, axis = 0)
        #print('maxTerm: {0}'.format(maxTerm))
        expMinusMax = self.exponentMatrix - maxTerm
        #print('expMinusMax: {0}'.format(expMinusMax))
        exponentiatedMatrix = np.exp(expMinusMax)
        #print('exponentiatedMatrix: {0}'.format(exponentiatedMatrix))
        collapsed = np.sum(exponentiatedMatrix, axis = 0)
        #print('collapsed: {0}'.format(collapsed))
        combined = maxTerm + np.log(collapsed)
        #print('combined: {0}'.format(combined))
        LL = - np.mean(combined)
        #print('Final LL: {0}'.format(LL))
        return LL
    
    def generate_predictions(self, points):
        topPredictions = np.argmin(self.find_test_distances(self.trained, points), axis = 0)
        #print('topPredictions has shape: {0}. Example labels: {1}'.format(topPredictions.shape, topPredictions[:5]))
        return topPredictions
        

class exemplar_model(classification_model):
    def __init__(self, train, trainLabels, test, testLabels, fit_beta = False, beta = 1):
        classification_model.__init__(self, train, trainLabels, test, testLabels)
        # make function below
        if fit_beta == False:
            self.beta = beta
        else:
            self.betas = np.logspace(-3, 1, 100)
            self.beta = self.fit_beta_CV()

        self.exemplarArray = self.form_exemplar_array(self.categories, self.train, self.trainLabels)
        
        self.residueArray, self.maxArrays, self.summedExponents = self.form_distance_arrays(self.exemplarArray,
                                                                                           self.test, self.beta)
        
        self.remainder, self.finalMax = self.create_exponent_matrix(self.testLabels, 
                                                                    self.maxArrays, 
                                                                    self.residueArray)
        
        self.LL = self.score_model_LogSumExp(self.finalMax, self.remainder)
        self.predictions = self.generate_predictions(self.exemplarArray, self.test)
        
        self.accuracy = self.get_accuracy(self.predictions, self.testLabels)
        
    def form_exemplar_array(self, categories, train, trainLabels):
        """form a numCategories-length array with each element an exemplar distance matrix"""
        exemplarArray = []
        for cat in categories:
            exemplars = train[np.where(trainLabels == cat)[0]]
            print_shape_fn('cat {0} exemplars'.format(cat), exemplars)
            
            exemplarArray.append(exemplars)
        exemplarArray = np.array(exemplarArray)
        print_shape_fn('exemplarArray', exemplarArray)
        
        return exemplarArray
    
    def form_distance_arrays(self, exemplarArray, testPoints, beta):
        residueArray = []
        maxArray = []
        summedExponents = []
        for cat, exemplars in enumerate(exemplarArray):
            #NB negation done here
            testDistances = - beta * self.find_test_distances(exemplars, testPoints)
            print_shape_fn('cat {0} distances'.format(cat), testDistances)
            maxTerm = np.max(testDistances, axis = 0)
            print_shape_fn('maxTerm', maxTerm)
            residueArray.append(np.sum(np.exp(testDistances - maxTerm), axis = 0)) # residues appended
            maxArray.append(maxTerm)
            summedExponents.append(np.sum(np.exp(testDistances), axis = 0))
            # here: might have to loop through
        residueArray = np.array(residueArray)
        maxArrays = np.array(maxArray)
        summedExponents = np.array(summedExponents)
        print_shape_fn('residueArray', residueArray)
        print_shape_fn('maxArrays', maxArrays)
        
        return residueArray, maxArrays, summedExponents
 
    def create_exponent_matrix(self, testLabels, maxArrays, residueArray):
        """returns negated matrix"""
        candidateIdx = np.squeeze(testLabels), np.arange(len(testLabels))
        #print(candidateIdx)
        candidateMax = maxArrays[candidateIdx]
        print_shape_fn('candidateMax', candidateMax)
        candidateResidue = residueArray[candidateIdx]
        print_shape_fn('candidateResidue', candidateResidue)
        maxMatrix = maxArrays - candidateMax # subtraction, as not exponentiated yet
        print_shape_fn('maxMatrix', maxMatrix)
        finalMax = np.max(maxMatrix, axis = 0)
        print_shape_fn('finalMax', finalMax)
        maxMinusMax = maxMatrix - finalMax
        print_shape_fn('maxMinusMax', maxMinusMax)
        residueMatrix = residueArray / candidateResidue # exp done and sorted
        print_shape_fn('residueMatrix', residueMatrix)
        #print('exponent matrix has shape: {0}'.format(exponentMatrix.shape))
        remainder = residueMatrix * np.exp(maxMinusMax)
        print_shape_fn('remainder', remainder)
        return remainder, finalMax 
    
    def score_model_LogSumExp(self, finalMax, remainder):
         # should be cats by samples
        combined = finalMax + np.log(np.sum(remainder, axis = 0))
        
        LL =  - np.mean(combined)
        return LL
    
    
    def fit_beta_CV(self):
        
        if self.numTrainingSamples <= 2:
            return 1
        elif self.numTrainingSamples == 4:
            nSplits = 4
        else:
            nSplits = 5 
        betaResults = np.empty((self.betas.shape[0], nSplits))
        kf = KFold(n_splits = nSplits, shuffle = True)
        fold = 0
        for train_index, test_index in kf.split(self.train):
            XTrain, XTest = self.train[train_index], self.train[test_index]
            YTrain, YTest = self.trainLabels[train_index], self.trainLabels[test_index]
            
            print_shape_fn('XTrain', XTrain)
            print_shape_fn('XTest', XTest)
            print_shape_fn('YTrain', YTrain)
            print_shape_fn('YTest', YTest)
            
            for i, beta in enumerate(self.betas):
                exemplarArray = self.form_exemplar_array(self.categories, XTrain, YTrain)
                residueArray, maxArrays, summedExponents = self.form_distance_arrays(exemplarArray,
                                                                                               XTest, beta)

                remainder, finalMax = self.create_exponent_matrix(YTest, 
                                                                        maxArrays, 
                                                                        residueArray)

                betaResults[i, fold] = self.score_model_LogSumExp(finalMax, remainder)
            fold += 1
        cvScores = np.mean(betaResults, axis = 1)
        bestBeta = self.betas[np.argmax(cvScores)]
        #print('best beta is: {0}. \n {1} \n {2}'.format(bestBeta, self.betas, cvScores))
        #print('bestBeta: {0}'.format(bestBeta))
        return bestBeta
    
# rebuild below   
    def generate_predictions(self, exemplarArray, points):
        _, _, summedExponentDistances = self.form_distance_arrays(exemplarArray, points, self.beta)
        return np.argmax(summedExponentDistances, axis = 0)
              

In [8]:
if fullCalc:
    data = np.load('{0}/simulated_data_nc_{1}_nd_{2}_ndis_{3}_it_{4}.npz'.format(dataDir,                                                                                                           
                                                 nc,
                                                 nd,
                                                 numDistributions, it))


    masterTrainArray = data["trainArray"]
    masterTrainLabels = data["trainLabels"]
    testArray = data["testArray"]
    testLabels = data["testLabels"]

    if beta:
        betaEx = exemplar_model(masterTrainArray, masterTrainLabels, testArray, testLabels,
                               fit_beta = True)
        bestBeta = betaEx.beta
    else:
        bestBeta = 1
        
    for ns in enumerate(np.int_(numSamples)):
        PTSavePath = '{0}/prototype_mean_accuracy_nc_{1}_nd_{2}_ns_{3}_ndis_{4}_it_{5}.npy'.format(saveDir,                                                                                                           
                                                                                 nc, nd, ns,
                                                                                 numDistributions, it)
        
        EXSavePath = '{0}/exemplar_mean_accuracy_nc_{1}_nd_{2}_ns_{3}_ndis_{4}_beta_{5}_it_{6}.npy'.format(saveDir,                                                                                                           
                                                                                 nc, nd, ns,
                                                                                 numDistributions, beta, it)
        if os.path.exists(EXSavePath):
            os.remove(EXSavePath)
        if os.path.exists(PTSavePath):
            os.remove(PTSavePath)
        
        
        trainArray, trainLabels = sample_from_data(masterTrainArray, masterTrainLabels, ns)

        pt = prototype_model(trainArray, trainLabels, testArray, testLabels)

        ex = exemplar_model(trainArray, trainLabels, testArray, testLabels, fit_beta = False, beta = bestBeta)

        np.save(PTSavePath, pt.accuracy)


        np.save(EXSavePath, ex.accuracy)



 For beta training: True
   Num components: 1
    
 dataset 0

      Num dimensions: 1
           bestBeta is: 1.291549665014884

      Num dimensions: 2
           bestBeta is: 4.328761281083062

      Num dimensions: 4
           bestBeta is: 2.0565123083486534

      Num dimensions: 8
           bestBeta is: 1.5556761439304723

      Num dimensions: 16
           bestBeta is: 0.7390722033525783

      Num dimensions: 32
           bestBeta is: 0.6734150657750828

      Num dimensions: 64
           bestBeta is: 0.7390722033525783

      Num dimensions: 128
           bestBeta is: 0.31992671377973847

      Num dimensions: 256
           bestBeta is: 0.1261856883066021

      Num dimensions: 512
           bestBeta is: 0.04534878508128584

      Num dimensions: 1024
           bestBeta is: 0.021544346900318846
[[0.02275 0.0415  0.04125 0.04175 0.0495  0.0485  0.05    0.05    0.05
  0.05    0.05   ]
 [0.025   0.0445  0.03175 0.04625 0.04775 0.04875 0.05    0.05    0.05
  0.05    0.05

           bestBeta is: 7.56463327554629

      Num dimensions: 2
           bestBeta is: 3.2745491628777317

      Num dimensions: 4
           bestBeta is: 2.0565123083486534

      Num dimensions: 8
           bestBeta is: 1.291549665014884

      Num dimensions: 16
           bestBeta is: 0.6135907273413176

      Num dimensions: 32
           bestBeta is: 1.7073526474706922

      Num dimensions: 64
           bestBeta is: 1.291549665014884

      Num dimensions: 128
           bestBeta is: 0.31992671377973847

      Num dimensions: 256
           bestBeta is: 0.11497569953977368

      Num dimensions: 512
           bestBeta is: 0.04132012400115339

      Num dimensions: 1024
           bestBeta is: 0.023644894126454083
[[0.12175 0.137   0.1635  0.1695  0.1875  0.196   0.19975 0.2     0.2
  0.2     0.2    ]
 [0.12875 0.1685  0.166   0.181   0.1935  0.1985  0.2     0.2     0.2
  0.2     0.2    ]
 [0.1195  0.1685  0.1755  0.19275 0.1955  0.199   0.2     0.2     0.2
  0.2     0.2   

           bestBeta is: 9.111627561154895

      Num dimensions: 2
           bestBeta is: 4.328761281083062

      Num dimensions: 4
           bestBeta is: 1.873817422860385

      Num dimensions: 8
           bestBeta is: 2.0565123083486534

      Num dimensions: 16
           bestBeta is: 0.7390722033525783

      Num dimensions: 32
           bestBeta is: 0.9770099572992257

      Num dimensions: 64
           bestBeta is: 1.291549665014884

      Num dimensions: 128
           bestBeta is: 0.29150530628251786

      Num dimensions: 256
           bestBeta is: 0.1261856883066021

      Num dimensions: 512
           bestBeta is: 0.05462277217684343

      Num dimensions: 1024
           bestBeta is: 0.021544346900318846
[[0.2275  0.241   0.28575 0.30425 0.32    0.34075 0.34975 0.35    0.35
  0.35    0.35   ]
 [0.22775 0.2765  0.29775 0.3245  0.33625 0.34725 0.35    0.35    0.35
  0.35    0.35   ]
 [0.22175 0.27125 0.31425 0.33725 0.3435  0.348   0.35    0.35    0.35
  0.35    0.35

           bestBeta is: 4.328761281083062

      Num dimensions: 2
           bestBeta is: 4.750810162102798

      Num dimensions: 4
           bestBeta is: 2.9836472402833403

      Num dimensions: 8
           bestBeta is: 1.1768119524349991

      Num dimensions: 16
           bestBeta is: 0.8111308307896873

      Num dimensions: 32
           bestBeta is: 1.873817422860385

      Num dimensions: 64
           bestBeta is: 2.4770763559917115

      Num dimensions: 128
           bestBeta is: 0.26560877829466867

      Num dimensions: 256
           bestBeta is: 0.10476157527896651

      Num dimensions: 512
           bestBeta is: 0.05462277217684343

      Num dimensions: 1024
           bestBeta is: 0.023644894126454083
[[0.3075  0.34425 0.414   0.41325 0.453   0.48725 0.49775 0.5     0.5
  0.5     0.5    ]
 [0.298   0.39025 0.43375 0.452   0.47825 0.49725 0.5     0.5     0.5
  0.5     0.5    ]
 [0.29425 0.39075 0.44275 0.47375 0.48775 0.498   0.5     0.5     0.5
  0.5     0.5  

           bestBeta is: 10.0

      Num dimensions: 2
           bestBeta is: 1.1768119524349991

      Num dimensions: 4
           bestBeta is: 1.4174741629268062

      Num dimensions: 8
           bestBeta is: 1.4174741629268062

      Num dimensions: 16
           bestBeta is: 10.0

      Num dimensions: 32
           bestBeta is: 5.72236765935022

      Num dimensions: 64
           bestBeta is: 1.1768119524349991

      Num dimensions: 128
           bestBeta is: 0.24201282647943834

      Num dimensions: 256
           bestBeta is: 0.15199110829529347

      Num dimensions: 512
           bestBeta is: 0.04534878508128584

      Num dimensions: 1024
           bestBeta is: 0.023644894126454083
[[0.438   0.4555  0.521   0.54425 0.58725 0.636   0.64675 0.65    0.65
  0.65    0.65   ]
 [0.426   0.4955  0.556   0.597   0.62375 0.645   0.64975 0.65    0.65
  0.65    0.65   ]
 [0.429   0.4975  0.56825 0.62125 0.63375 0.64775 0.65    0.65    0.65
  0.65    0.65   ]
 [0.4445  0.51225 0.

...

           bestBeta is: 3.94420605943766

      Num dimensions: 2
           bestBeta is: 2.9836472402833403

      Num dimensions: 4
           bestBeta is: 1.7073526474706922

      Num dimensions: 8
           bestBeta is: 1.0722672220103242

      Num dimensions: 16
           bestBeta is: 0.7390722033525783

      Num dimensions: 32
           bestBeta is: 7.56463327554629

      Num dimensions: 64
           bestBeta is: 0.9770099572992257

      Num dimensions: 128
           bestBeta is: 0.31992671377973847

      Num dimensions: 256
           bestBeta is: 0.18307382802953698

      Num dimensions: 512
           bestBeta is: 0.05462277217684343

      Num dimensions: 1024
           bestBeta is: 0.023644894126454083
[[0.6375  0.66475 0.78525 0.77575 0.86025 0.9035  0.93525 0.91625 0.90925
  0.91875 0.93325]
 [0.68475 0.7095  0.8675  0.869   0.90675 0.947   0.919   0.946   0.95475
  0.923   0.954  ]
 [0.695   0.724   0.8725  0.909   0.94075 0.97325 0.987   0.96475 0.99675
  0.

           bestBeta is: 2.2570197196339215

      Num dimensions: 2
           bestBeta is: 3.94420605943766

      Num dimensions: 4
           bestBeta is: 1.873817422860385

      Num dimensions: 8
           bestBeta is: 1.0722672220103242

      Num dimensions: 16
           bestBeta is: 0.7390722033525783

      Num dimensions: 32
           bestBeta is: 10.0

      Num dimensions: 64
           bestBeta is: 0.8902150854450392

      Num dimensions: 128
           bestBeta is: 0.35111917342151344

      Num dimensions: 256
           bestBeta is: 0.11497569953977368

      Num dimensions: 512
           bestBeta is: 0.049770235643321115

      Num dimensions: 1024
           bestBeta is: 0.023644894126454083
[[0.614   0.653   0.759   0.7525  0.84225 0.87925 0.91925 0.88775 0.874
  0.8785  0.905  ]
 [0.66025 0.6865  0.82575 0.8375  0.88    0.939   0.90175 0.94575 0.9175
  0.89775 0.917  ]
 [0.693   0.70525 0.84625 0.88775 0.9165  0.9655  0.987   0.9615  0.98425
  0.98875 0.96175]


           bestBeta is: 4.750810162102798

      Num dimensions: 2
           bestBeta is: 2.4770763559917115

      Num dimensions: 4
           bestBeta is: 1.5556761439304723

      Num dimensions: 8
           bestBeta is: 1.291549665014884

      Num dimensions: 16
           bestBeta is: 0.8902150854450392

      Num dimensions: 32
           bestBeta is: 5.72236765935022

      Num dimensions: 64
           bestBeta is: 0.8111308307896873

      Num dimensions: 128
           bestBeta is: 0.4229242874389499

      Num dimensions: 256
           bestBeta is: 0.1261856883066021

      Num dimensions: 512
           bestBeta is: 0.05462277217684343

      Num dimensions: 1024
           bestBeta is: 0.023644894126454083
[[0.587   0.653   0.76125 0.72625 0.829   0.85175 0.9035  0.84125 0.8395
  0.8405  0.87025]
 [0.649   0.69575 0.8125  0.81625 0.8555  0.92325 0.874   0.915   0.892
  0.866   0.89325]
 [0.66075 0.717   0.8195  0.866   0.891   0.95975 0.98625 0.958   0.97075
  0.9755 

           bestBeta is: 10.0

      Num dimensions: 2
           bestBeta is: 2.9836472402833403

      Num dimensions: 4
           bestBeta is: 1.291549665014884

      Num dimensions: 8
           bestBeta is: 1.0722672220103242

      Num dimensions: 16
           bestBeta is: 0.6135907273413176

      Num dimensions: 32
           bestBeta is: 5.72236765935022

      Num dimensions: 64
           bestBeta is: 1.291549665014884

      Num dimensions: 128
           bestBeta is: 0.29150530628251786

      Num dimensions: 256
           bestBeta is: 0.11497569953977368

      Num dimensions: 512
           bestBeta is: 0.049770235643321115

      Num dimensions: 1024
           bestBeta is: 0.023644894126454083
[[0.55125 0.6075  0.7195  0.687   0.807   0.81675 0.87225 0.8015  0.802
  0.81125 0.8325 ]
 [0.635   0.67125 0.78825 0.7925  0.84275 0.897   0.8435  0.88825 0.86675
  0.83175 0.85725]
 [0.64775 0.6805  0.78725 0.8495  0.87425 0.9445  0.973   0.95425 0.94725
  0.96325 0.96175]


           bestBeta is: 5.21400828799969

      Num dimensions: 2
           bestBeta is: 4.750810162102798

      Num dimensions: 4
           bestBeta is: 1.873817422860385

      Num dimensions: 8
           bestBeta is: 0.8902150854450392

      Num dimensions: 16
           bestBeta is: 0.6734150657750828

      Num dimensions: 32
           bestBeta is: 0.6734150657750828

      Num dimensions: 64
           bestBeta is: 0.8902150854450392

      Num dimensions: 128
           bestBeta is: 0.3853528593710531

      Num dimensions: 256
           bestBeta is: 0.15199110829529347

      Num dimensions: 512
           bestBeta is: 0.049770235643321115

      Num dimensions: 1024
           bestBeta is: 0.023644894126454083
[[0.502   0.601   0.68075 0.6675  0.77625 0.77675 0.84975 0.75925 0.774
  0.7845  0.819  ]
 [0.59625 0.6625  0.7595  0.7755  0.8405  0.8795  0.8245  0.8655  0.8305
  0.80875 0.82225]
 [0.62025 0.67875 0.76125 0.83075 0.8575  0.937   0.962   0.93425 0.94725
  0.963